# --------------------------  CVA Avec intensité stochastique  ------------------------

In [13]:
import random 
import numpy as np
import matplotlib.pyplot as plt
import time

# B&S and Payoff Call with Monte Carlo

In [30]:
## Monte Carlo method for valuation of Financial product following B&S model
S0=100.0
k=100.0
T=1.0
r=0.05
sigma=0.20

def BSM(Option,t):
    S0,K,T,r,sigma=Option[0],Option[1],Option[2],Option[3],Option[4]
    N=int(1e4)
    W=np.random.standard_normal(N)
    St=S0*np.exp((r-0.5*sigma**2)*t+sigma*np.sqrt(t)*W)
    MonteCarloprice=np.sum(St)*np.exp(-r*t)/N
    return MonteCarloprice

def BSM_payoff(Option,t):
    S0,K,T,r,sigma=Option[0],Option[1],Option[2],Option[3],Option[4]
    N=int(1e4)
    W=np.random.standard_normal(N)
    St=S0*np.exp((r-0.5*sigma**2)*t+sigma*np.sqrt(t)*W)
    Mt=np.maximum(0,St-K)
    MonteCarloprice=np.sum(Mt)*np.exp(-r*t)/N
    return MonteCarloprice

BSM([S0,k,T,r,sigma],1.0)

100.1280668893981

# Espérance de l'instant de défaut

In [205]:
def Int_Rem(X,Y):
    x=0
    for i in range(len(X)):
        x+=X[i]*(Y[i+1]-Y[i])
    return x

def Instant_Defaut(Mod_int,T): # Mod_int est le modèle de B&S choisi pour l'intensité Lambda
    Tho,x=0,0
    m=100 # l'intervalle [0,T] est subdivisé en m instant
    size=10000 # on génère size valeurs de l'uniforme
    for j in range(m):
        Repr_temps=[T*j*i/m for i in range(size)]
        Intensite_j=[BSM(Mod_int,t) for t in  Repr_temps[1:]]# Génération d'intensités entre 0 et T*j/m
        Int=Int_Rem(Intensite_j,Repr_temps) # intégrale de l'intensité
        Uniforme=np.random.uniform(0,1,size)  # size valeurs de l'uniforme
        for i in range(size):
            if Uniforme[i]<=1-np.exp(-Int):
                Tho+=T*j/m
                x+=1
    return Tho/x

In [206]:
S0,K,T,r,sigma=100.0,105.0,1.0,0.05,0.2
Mod_int=[0.1,K,T,r,sigma]
Instant_Defaut(Mod_int,T)

0.4999999999994952

# Calcul de la CVA avec corrélation 

In [228]:
def CVA_corr(cor):
    T,sigmaS,sigmaL,K,r=1.0,0.2,0.2,100,0.05
    S0,Lamda0=100,1
    N=10000  # Tragectoire de MC
    M= 500  # Subdivision de l'intervalle de temps
    
    CVA=0
    Uniforme=np.random.uniform(0,1,N)
    for i in range(N):
        t,Int=0,0
        # Détermination de l'instant de défaut 
        while t<=T:
            t+=T/M
            #Calcul de St et Lambdat qui sont corrélés 
            V,W=np.random.multivariate_normal([0,0],[[1,cor],[cor,1]],1).T # 2 normales corrélées
            St=S0*np.exp((-0.5*sigmaS**2)*t+sigmaS*np.sqrt(t)*W)
            Lamdat=Lamda0*np.exp((-0.5*sigmaL**2)*t+sigmaL*np.sqrt(t)*V)
            Int+=Lamdat*T/M  #Intégrale de l'intensité
            # Détermination de l'instant de défaut
            if Uniforme[i]<=1-np.exp(-Int):
                Tho=t
                Xt=np.exp(-r*t)*np.maximum(0,St-K)
                break
        if t>T: # Le cas où il n'y a pas défaut
            Xt=0
        CVA+=Xt/N # CVA est un espérance du payoff à l'instant de défaut
    return CVA

Calcul de la CVA avec corrélation entre l'instant de défaut et le sous-jacent (On considère la corrélation nulle pour vérifier si nous obtenons la même valeur que la CVA_ind) :

In [229]:
start=time.time()
x=CVA_corr(0)
end=time.time()
print(end-start,x)

826.2333376407623 [2.86240139]


# Calcul de la CVA avec indépendance

In [230]:
def CVA_ind():
    T,sigmaS,sigmaL,K,r=1.0,0.2,0.2,100,0.05
    S0,Lamda0=100,1
    N=100000  # Tragectoire de MC
    M= 1000  # Subdivision de l'intervalle de temps
    
    Lamda=[[]]*N
    Int=[[]]*N  #Intégrale de l'intensité
    
    U=np.random.standard_normal(N)  # N trajectoire de MC
    for i in range(N):
        x=Lamda0*np.exp((-0.5*sigmaL**2)*T/M+sigmaL*np.sqrt(T/M)*U[i])
        Lamda[i].append(x)
        Int[i].append(x*T/M)
        for j in range(1,M):
            v=Lamda0*np.exp((-0.5*sigmaL**2)*(j+1)*T/M+sigmaL*np.sqrt((j+1)*T/M)*U[i])
            Lamda[i].append(v)
            Int[i].append(Int[i][j-1]+v*T/M)
    CVA=0
    for j in range(M):
        #Fonction densité de l'instant de défaut à tj
        densite=sum([Lamda[i][j]*np.exp(-Int[i][j]) for i in range(N)])/N
        #CVA en Riemann
        CVA+=BSM_payoff([S0,K,T,r,sigmaS],(j+1)*T/M)*densite*T/M
    return CVA

Calcul de la CVA avec indépendance entre l'instant de défaut et le sous-jacent:

In [231]:
start=time.time()
x=CVA_ind()
end=time.time()
print(end-start,x)


1203.6130855083466 3.804431789549676


In [234]:
print('le rapport entre les deux valeurs',2.86240139/3.804431789549676)

le rapport entre les deux valeurs 0.7523860456278065
